# This is a demo + test notebook for GraphTranslatorModule

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from GraphTranslatorModule import GraphTranslatorModule
from pytorch_lightning import Trainer
import torch
from torch.nn import functional as F
from pl_bolts.datasets import DummyDataset
import random
import numpy as np

## A. This section is a primer for LightningModule

### This is a lightning module instead of a torch.nn.Module. A LightningModule is equivalent to a pure PyTorch Module except it has added functionality. However, you can use it EXACTLY the same as you would a PyTorch Module.

net = GraphTranslatorModule()
x = torch.randn(1, 1, 3, 3)
out = net(x)

trainer = Trainer(max_epochs=3)
trainer.fit(net, torch.utils.data.DataLoader(DummyDataset((1,3,3),(1,), num_samples=100), num_workers=8))

trainer.test(net, torch.utils.data.DataLoader(DummyDataset((1,3,3),(1,), num_samples=10), num_workers=8))

## B. Dimension checks for the model functions

### Test model with Dummy data

In [3]:
# class DummyClassificationDataset():
#     def __init__(self, n_nodes, n_len, e_len, c_len, num_samples: int = 100):
#         self.n_nodes = n_nodes
#         self.n_len = n_len
#         self.e_len = e_len
#         self.c_len = c_len
#         self.num_samples = num_samples
    
#     def __len__(self):
#         return self.num_samples

#     def __getitem__(self, idx: int):
#         edges = torch.randn(self.n_nodes, self.n_nodes, self.e_len)
#         nodes = torch.randn(self.n_nodes, self.n_len)
#         context = torch.randn(self.c_len)
#         y = torch.randn(self.n_nodes, self.n_nodes, self.e_len)
#         return (edges.contiguous(), nodes.contiguous(), context.contiguous(), context.contiguous(), y.contiguous())

# n_nodes = 5
# n_len = 2
# e_len = 3
# c_len = 1
# model = GraphTranslatorModule(num_nodes=n_nodes, node_feature_len=n_len, edge_feature_len=e_len, context_len=c_len)
# dataset = DummyClassificationDataset(n_nodes, n_len, e_len, c_len)
# dl = torch.utils.data.DataLoader(dataset, num_workers=8, batch_size=3)

# trainer = Trainer(max_epochs=3, log_every_n_steps=1, flush_logs_every_n_steps=1)
# trainer.fit(model, dl)

### Test model with example graph and timestamp data

In [13]:
from Reader import read_data

class DataSplit():
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx: int):
        return self.data[idx]

class SingleDataset():
    def __init__(self, data_path: str = 'data/routines_1028/sample.json', classes_path: str = 'data/routines_1028/classes.json', test_perc = 0.2):
        self._alldata = read_data(data_path = data_path, classes_path = classes_path)
        print(len(self._alldata),' examples found in dataset.')
        edges, nodes, context_curr, context_query, y = self._alldata[0]
        self.n_nodes = edges.size()[1]
        self.n_len = nodes.size()[1]
        self.e_len = edges.size()[-1]
        self.c_len = context_curr.size()[0]
        random.shuffle(self._alldata)
        num_test = int(round(test_perc*len(self._alldata)))
        self.test = DataSplit(self._alldata[:num_test])
        self.train = DataSplit(self._alldata[num_test:])
        print(len(self.train),' examples in train split.')
        print(len(self.test),' examples in test split.')
        losses = torch.Tensor([0])
        for d in self.train:
            r = torch.randn(self.n_nodes, self.n_nodes, self.e_len)
            random_out = torch.nn.Sigmoid()(r)
            losses = losses + torch.nn.BCELoss()(random_out, d[4])
        print("random loss : ", losses/len(self.train))

d = SingleDataset()
dl_train = torch.utils.data.DataLoader(d.train, num_workers=8, batch_size=10)
dl_test = torch.utils.data.DataLoader(d.test, num_workers=8, batch_size=10)
model = GraphTranslatorModule(num_nodes=d.n_nodes, node_feature_len=d.n_len, edge_feature_len=d.e_len, context_len=d.c_len)

trainer = Trainer(max_epochs=15)
trainer.fit(model, dl_train)
trainer.test(model, dl_test)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name          | Type       | Params
---------------------------------------------
0 | mlp_influence | Sequential | 2.3 K 
1 | mlp_update    | Sequential | 943   
2 | bce           | BCELoss    | 0     
---------------------------------------------
3.2 K     Trainable params
0         Non-trainable params
3.2 K     Total params
0.013     Total estimated model params size (MB)


1543  examples found in dataset.
1234  examples in train split.
309  examples in test split.
random loss :  tensor([0.8063])
Testing:  68%|██████▊   | 21/31 [00:00<00:00, 104.74it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.0249982587993145}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 31/31 [00:00<00:00, 98.66it/s] 


[{'test_loss': 0.0249982587993145}]